# Imports and functions:

In [1]:
import numpy as np
import os# to handle files in the folder
from RsInstrument import *#<- both this or the above command works to import RsInstrument even if spyder complins about this one
from time import sleep
import matplotlib.pyplot as plt
import re

In [2]:
from pathlib import Path
import sys
home = str(Path.home())
sys.path.insert(0, home+"//Desktop//python_libraries//VISA")
import pyvisa

In [3]:
%run ./command_lybrary.ipynb #change instrument adress in command_lybrary.ipynb if necessary

In [4]:
def purge(inst):
    i = 0
    while True:
        try:
            inst.read_bytes(1)
            i += 1
        except:
            break
        
    return i

In [5]:
def clean_term(x):
    aux = repr(x)
    aux = re.sub('\\\\n\\\\.*', '', aux)[1:]
    return aux

In [6]:
filepath = "C:/Users/Administrator/Desktop/cavity_magnonics_TP4/data/"

def measurement(freq, f_pts, volt, v_pts, filename, avg = 1):
    
    PcFile=filepath+filename
    
    set_start_freq(freq[0])
    set_stop_freq(freq[1])
    set_sweep_points(f_pts)
    volt_sweep = np.linspace(volt[0], volt[1], v_pts)
    
    znb.write_str('INITiate1:CONTinuous 0')
    average(avg)
    
    isFile = os.path.isfile(PcFile)
    if isFile:
        print('file ', filename,' already exists, chose another name')
    else:
        
        #purge(com3)
        #purge(com4)
        purge(com5)
        
        #com3.write('CURR040')
        #com4.write('CURR040') 
        com5.write('CURR050')
            
        #com3.read_bytes(3)
        #com4.read_bytes(3) #necessary to get rid of the "OK\r"
        com5.read_bytes(3)
        
        #com3.write('VOLT010')
        #com4.write('VOLT010') 
        com5.write('VOLT010')
            
        #com3.read_bytes(3)
        #com4.read_bytes(3) #necessary to get rid of the "OK\r"
        com5.read_bytes(3)
        
        
        Data_file = open(PcFile, "w")
        Data_file.write('Time;Display;Averages;Field[mT];Frequencies[Hz];RealandImaginary\n')
        
        for volt in volt_sweep:
            
            #print(field)
            #com3.write('VOLT' + f'{int(volt*10):03}')
            #com4.write('VOLT' + f'{int(volt*10):03}') 
            com5.write('VOLT' + f'{int(volt*10):03}')
            
            #com3.read_bytes(3)
            #com4.read_bytes(3) #necessary to get rid of the "OK\r"
            com5.read_bytes(3)
            #set_current(field)
            
            sleep(1)
            #disp3 = com3.query("GETD")
            #com3.read_bytes(3)
            #disp4 = com4.query("GETD")
            #com4.read_bytes(3)
            disp5 = com5.query("GETD")
            com5.read_bytes(3)
            
            field = (22015.862*float(clean_term(lockin.query('MAG.')))-28402.895)/10 #field in mT input 4 (+) and 2 (gnd) output 1 and 3
            znb.write_str("INITiate:IMMediate:ALL")
            znb.query_opc()
            
            freq_str=znb.query_str('CALC:DATA:STIM?')#get the string of the stimulus frequencies
            trace_str=znb.query_str('CALC1:DATA? SDAT')# the string is written as Re(S_point1),Im(S_point1),Re(S_point2)....
            time=znb.query_str('SYSTem:TIME?').replace(",",":")
            
            #Data_file.write(str(time)+";"+str(disp3) +","+str(disp4)+","+str(disp5)+";"+str(avg)+";"+str(field)+";"+str(freq_str)+";"+str(trace_str)+"\n")
            Data_file.write(str(time)+";"+str(disp5)+";"+str(avg)+";"+str(field)+";"+str(freq_str)+";"+str(trace_str)+"\n")
            
    Data_file.close()
    #com3.write("CURR000")
    #com4.write("CURR000")
    com5.write("CURR000")
    #com3.read_bytes(3)
    #com4.read_bytes(3) #necessary to get rid of the "OK\r"
    com5.read_bytes(3)
    #close()# close communicatin with VNA
    print("file has succesfully been created :)")

# ZNB prep:

In [7]:
com_prep()

VISA Manufacturer: National Instruments


In [8]:
com_check()

Hello, I am Rohde-Schwarz,ZNB40-4Port,1311601084101762,3.32


# Manson prep:

In [34]:
rm = pyvisa.ResourceManager()
rm.list_resources('?*')

('USB0::0x0A2D::0x001B::09295879::RAW',
 'USB0::0x0AAD::0x01C0::101762::INSTR',
 'ASRL1::INSTR',
 'ASRL5::INSTR')

In [16]:
com4 = rm.open_resource('ASRL4::INSTR')
com4.write_termination = '\r'
com4.read_termination = '\r'

In [12]:
com5 = rm.open_resource('ASRL5::INSTR')
com5.write_termination = '\r'
com5.read_termination = '\r'

In [15]:
lockin = rm.open_resource('USB0::0x0A2D::0x001B::09295879::RAW')
print(float(clean_term(lockin.query('MAG.'))))

0.0001


# Data aquisition:

In [16]:
measurement([1.85e9, 2.85e9], 20000, [20,60], 400,"yig_quarterturn2.csv")

file has succesfully been created :)


# Testing:

In [ ]:
field = (22015.862*float(clean_term(lockin.query('MAG.')))-28402.895)/10
print(float(field))

In [30]:
purge(com5)

18

In [22]:
com5.write("VOLT100")

8

In [ ]:
avg =10
average(avg)
sweep_time=float(znb.query_str('SWEep:TIME?'))
znb.write_str('INITiate1:CONTinuous 1')
sleep(sweep_time*avg*1.25)
znb.write_str('INITiate1:CONTinuous 0')

In [ ]:
znb.write_str("INITiate:IMMediate:ALL")
znb.query_opc()
print("done!")

In [ ]:
freq_str=znb.query_str('CALC:DATA:STIM?')#get the string of the stimulus frequencies
freq_tup = tuple(map(float, freq_str.split(',')))  # Convert the received string into a tuple
trace_str=znb.query_str('CALC1:DATA? SDAT')# get the string of the measure real and immaginary part of the measured parameter
                                            # the list is written as Re(S_point1),Im(S_point1),Re(S_point2)....
trace_tup = tuple(map(float, trace_str.split(',')))  # Convert the received string into a tuple
Re_trace_tup = trace_tup[::2]
Im_trace_tup = trace_tup[1::2]

In [ ]:
print(len(freq_tup), len(Re_trace_tup), len(Im_trace_tup))

In [ ]:
plt.close()
plt.plot(freq_tup, Re_trace_tup, label = "Real part")
plt.plot(freq_tup, Im_trace_tup, label = "Imaginary part")
plt.legend()
plt.show()

In [ ]:
amp_tup = tuple((np.array(Re_trace_tup)**2 + np.array(Im_trace_tup)**2)**0.5)
plt.close()
plt.plot(freq_tup, amp_tup)
plt.show()

In [ ]:
for i in range(990, 1010):
    print(f'{i:03}')

In [33]:
com4.write("CURR000")
com5.write("CURR000")

8

In [31]:
com4.write("VOLT000")
com5.write("VOLT000")

8

In [ ]:
field = 1.2

com3.write('CURR' + f'{int(field*10):03}')
com4.write('CURR' + f'{int(field*10):03}')
com5.write('CURR' + f'{int(field*10):03}')
            
com3.read_bytes(3)
com4.read_bytes(3) #necessary to get rid of the "OK\r"
com5.read_bytes(3)
            
sleep(30)
resp3 = com3.query("GETD")[5:9]
com3.read_bytes(3)
            
znb.write_str("INITiate:IMMediate:ALL")
znb.query_opc()
            
freq_str=znb.query_str('CALC:DATA:STIM?')#get the string of the stimulus frequencies
trace_str=znb.query_str('CALC1:DATA? SDAT')# the string is written as Re(S_point1),Im(S_point1),Re(S_point2)....
time=znb.query_str('SYSTem:TIME?').replace(",",":")

In [ ]:
import pandas as pd

In [ ]:
filepath = "C:/Users/Administrator/Desktop/cavity_magnonics_TP4/OneDrive_1_4-22-2024/"
filename = "calbration_curve_input2and3_output4to1.csv"
PcFile=filepath+filename
data = pd.read_csv(PcFile,sep=',')
oe = data["field[Oe]"]
r = data["R [V]"]
print(np.polyfit(r, oe, 1))

In [ ]:
filepath = "C:/Users/Administrator/Desktop/cavity_magnonics_TP4/OneDrive_1_4-22-2024/"
filename = "calibration_curve_input1and2_output_3and4.csv"
PcFile=filepath+filename
data = pd.read_csv(PcFile,sep=',')
oe = data["field[Oe]"]
r = data["R [V]"]
print(np.polyfit(r, oe, 1))

In [ ]:
filepath = "C:/Users/Administrator/Desktop/cavity_magnonics_TP4/OneDrive_1_4-22-2024/"
filename = "calibration_curve_input2and4_output3to1.csv"
PcFile=filepath+filename
data = pd.read_csv(PcFile,sep=',')
oe = data["field[Oe]"]
r = data["R [V]"]
print(np.polyfit(r, oe, 1))

In [ ]:
def set_current(curr):
    com3.write("VOLT200")
    com4.write("VOLT200")
    com5.write("VOLT200")
    com3.read_bytes(3)
    com4.read_bytes(3) #necessary to get rid of the "OK\r"
    com5.read_bytes(3)
    
    com3.write("CURR" + f'{int(curr*10):03}')
    com4.write("CURR" + f'{int(curr*10):03}')
    com5.write("CURR" + f'{int(curr*10):03}')
    com3.read_bytes(3)
    com4.read_bytes(3) #necessary to get rid of the "OK\r"
    com5.read_bytes(3)
    
    sleep(0.5)
    v3 = com3.query("GETD")[:3]
    com3.read_bytes(3)
    com3.write("VOLT"+v3)
    com3.read_bytes(3)
    v4 = com4.query("GETD")[:3]
    com4.read_bytes(3)
    com4.write("VOLT"+v4)
    com4.read_bytes(3)
    v5 = com5.query("GETD")[:3]
    com5.read_bytes(3)
    com5.write("VOLT"+v5)
    com5.read_bytes(3)